<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [26]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2023-11-05 18:00:13--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2023-11-05 18:00:14--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfaeb24d9b3032c88f3e3be0dad.dl.dropboxusercontent.com/cd/0/get/CG-9Nt0wwHZ8dLIRr1jYOeK_AopuLrbwDwgVY2b3qzuI6xziFgmEE-w4uPDsk4XOZ9Cs43-JwjD-BXx5NAlTWK8oI4ROsT61tWCPE1BMiZbxqY7oFS92e1wFzygVuaoQ2EY/file?dl=1# [following]
--2023-11-05 18:00:14--  https://ucfaeb24d9b3032c88f3e3be0dad.dl.dropboxusercontent.com/cd/0/get/CG-9Nt0wwHZ8dLIRr1jYOeK_AopuLrbwDwgVY2b3qzuI6xziFgmEE-w4uPDsk4XOZ9Cs43-JwjD-BXx5NAlTWK8oI4ROsT61tWCPE1BMiZbxqY7oFS9

In [27]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`).

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [28]:
import numpy as np

def msle(ys, ps):
    assert len(ys) == len(ps)
    return ((np.log(1 + ys) - np.log(1 + ps))**2).mean()


def mse_loss(ys, ps):
    assert len(ys) == len(ps)
    return ((ys - ps)**2).mean()

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [29]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################

import pandas as pd

mieszkania = pd.read_csv('mieszkania.csv')
mieszkania_test = pd.read_csv('mieszkania_test.csv')

mean_price = mieszkania['cena'].mean()
prices = np.full(mieszkania['cena'].shape, mean_price)

print(mean_price)
print(msle(prices, mieszkania['cena']))

507919.49
0.3915253538257009


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [30]:
#############################################
# TODO: Find this constant and compute msle #
#############################################

pre_exp = (np.log(mieszkania['cena']+1)).mean()
min_constant = np.exp(pre_exp)-1

print(min_constant)
print(msle(np.ones(mieszkania['cena'].shape) * min_constant, mieszkania['cena']))

431435.4915823427
0.3648896122133589


Now, let's implement a standard linear regression model.

In [31]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################
from sklearn import preprocessing

# xs is array of (number of apartments X number of features of each apartment)
xs = np.array(mieszkania.loc[:, mieszkania.columns != 'cena'])

# ys is our array of actual prices we are trying to train our model to
ys = np.array(mieszkania['cena'])

# We are looking for optimal weights and biases for our given xs and features ys.
# So that weights * xs + biases = ys, shorter: wx + b = y.
# Since we have 200 apartments, they are our "obeservations" / input data
# and each apartment has 6 features that it "contains", so we will have
# 6 different weights we want to optimize, one for each feature.
weights = np.zeros(xs[0].shape)
biases = np.zeros(ys.shape)
lr = 1e-7
n_epochs = 10000

# Label encoding so that in xs array we have only numerical values
# It codes each string label, since string labels are only in 1. column
# we use the encoder only on first column, with an unique value assigned to it.
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(xs[:, 1])
xs[:, 1] = labelencoder.transform(xs[:, 1])
xs = xs.astype(np.float64)

# Function to calculate our current prediction of weights and biases
def predict(w, b, xs=xs):
  return np.array([xs @ w + b]).squeeze()

# Just function to compute MSE Loss for our current prediction of weights and biases
def evaluate(w, b, xs=xs, ys=ys):
  return mse_loss(ys, predict(w, b, xs))

losses = [evaluate(weights, biases)]
xs_len = len(xs)

for i in range(n_epochs):

  ps = predict(weights, biases)

  # Gradient descent for weights and biases, derivative of MSE Loss function
  weights -= lr * 2 / xs_len * (xs.T @ (ps - ys))
  biases -= lr * 2 / xs_len * (ps - ys)

  loss = evaluate(weights, biases)
  losses.append(loss)

  print(f'Iter: {i:>3} Loss: {loss:8.8f}')

# Computing MSLE Loss for test data
xs_test = np.array(mieszkania_test.loc[:, mieszkania_test.columns != 'cena'])
xs_test[:, 1] = labelencoder.transform(xs_test[:, 1])
xs_test = xs_test.astype(np.float64)
ys_test = np.array(mieszkania_test['cena'])
print("MSLE Loss for test data: ", msle(ys_test, predict(weights, biases, xs_test)))

Streaming output truncated to the last 5000 lines.
Iter: 5001 Loss: 17934697239.68767929
Iter: 5002 Loss: 17932869337.75494766
Iter: 5003 Loss: 17931042501.84181595
Iter: 5004 Loss: 17929216731.32647705
Iter: 5005 Loss: 17927392025.58750534
Iter: 5006 Loss: 17925568384.00384140
Iter: 5007 Loss: 17923745805.95478821
Iter: 5008 Loss: 17921924290.82000351
Iter: 5009 Loss: 17920103837.97950745
Iter: 5010 Loss: 17918284446.81369781
Iter: 5011 Loss: 17916466116.70330429
Iter: 5012 Loss: 17914648847.02943420
Iter: 5013 Loss: 17912832637.17356873
Iter: 5014 Loss: 17911017486.51752090
Iter: 5015 Loss: 17909203394.44349289
Iter: 5016 Loss: 17907390360.33403397
Iter: 5017 Loss: 17905578383.57205582
Iter: 5018 Loss: 17903767463.54081345
Iter: 5019 Loss: 17901957599.62395096
Iter: 5020 Loss: 17900148791.20544815
Iter: 5021 Loss: 17898341037.66966629
Iter: 5022 Loss: 17896534338.40130234
Iter: 5023 Loss: 17894728692.78541946
Iter: 5024 Loss: 17892924100.20745468
Iter: 5025 Loss: 17891120560.05318451

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [32]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################
from sklearn import preprocessing

# xs is array of (number of apartments X number of features of each apartment)
xs = np.array(mieszkania.loc[:, mieszkania.columns != 'cena'])

# ys is our array of actual prices we are trying to train our model to
ys = np.array(mieszkania['cena'])

# We are looking for optimal weights and biases for our given xs and features ys.
# So that weights * xs + biases = ys, shorter: wx + b = y.
# Since we have 200 apartments, they are our "obeservations" / input data
# and each apartment has 6 features that it "contains", so we will have
# 6 different weights we want to optimize, one for each feature.
weights = np.zeros(xs[0].shape)
biases = np.zeros(ys.shape)
lr = 1e-7
n_epochs = 10000

# Label encoding so that in xs array we have only numerical values
# It codes each string label, since string labels are only in 1. column
# we use the encoder only on first column, with an unique value assigned to it.
labelencoder = preprocessing.LabelEncoder()
labelencoder.fit(xs[:, 1])
xs[:, 1] = labelencoder.transform(xs[:, 1])
xs = xs.astype(np.float64)

# Function to calculate our current prediction of weights and biases
def predict(w, b, xs=xs):
  return np.array([xs @ w + b]).squeeze()

# Just function to compute MSLE Loss for our current prediction of weights and biases
def evaluate(w, b, xs=xs, ys=ys):
  return msle(ys, predict(w, b, xs))

losses = [evaluate(weights, biases)]
xs_len = len(xs)

for i in range(n_epochs):

  ps = predict(weights, biases)

  # Gradient descent for weights and biases, derivative of MSLE Loss function
  weights -= lr * 2 / xs_len * (xs.T @ (ps - ys)) # FIX THIS DERIVATIVE
  biases -= lr * (-2) / xs_len * ((np.log(1 + ys) - np.log(1 + ps)) / (1 + ps))

  loss = evaluate(weights, biases)
  losses.append(loss)

  print(f'Iter: {i:>3} Loss: {loss:8.8f}')

# Computing MSLE Loss for test data
xs_test = np.array(mieszkania_test.loc[:, mieszkania_test.columns != 'cena'])
xs_test[:, 1] = labelencoder.transform(xs_test[:, 1])
xs_test = xs_test.astype(np.float64)
ys_test = np.array(mieszkania_test['cena'])
print("MSLE Loss for test data: ", msle(ys_test, predict(weights, biases, xs_test)))

Streaming output truncated to the last 5000 lines.
Iter: 5001 Loss: 0.08485371
Iter: 5002 Loss: 0.08483674
Iter: 5003 Loss: 0.08481977
Iter: 5004 Loss: 0.08480281
Iter: 5005 Loss: 0.08478586
Iter: 5006 Loss: 0.08476892
Iter: 5007 Loss: 0.08475199
Iter: 5008 Loss: 0.08473506
Iter: 5009 Loss: 0.08471815
Iter: 5010 Loss: 0.08470124
Iter: 5011 Loss: 0.08468433
Iter: 5012 Loss: 0.08466744
Iter: 5013 Loss: 0.08465055
Iter: 5014 Loss: 0.08463368
Iter: 5015 Loss: 0.08461681
Iter: 5016 Loss: 0.08459994
Iter: 5017 Loss: 0.08458309
Iter: 5018 Loss: 0.08456624
Iter: 5019 Loss: 0.08454941
Iter: 5020 Loss: 0.08453257
Iter: 5021 Loss: 0.08451575
Iter: 5022 Loss: 0.08449894
Iter: 5023 Loss: 0.08448213
Iter: 5024 Loss: 0.08446533
Iter: 5025 Loss: 0.08444854
Iter: 5026 Loss: 0.08443176
Iter: 5027 Loss: 0.08441498
Iter: 5028 Loss: 0.08439822
Iter: 5029 Loss: 0.08438146
Iter: 5030 Loss: 0.08436471
Iter: 5031 Loss: 0.08434796
Iter: 5032 Loss: 0.08433123
Iter: 5033 Loss: 0.08431450
Iter: 5034 Loss: 0.084297

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [43]:
################################################
# TODO: Implement the feature engineering part #
################################################

In [42]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################